In [1]:
import numpy as np
import math
import sympy
import pandas as pd

In [2]:
df = pd.read_csv('data1.csv')
df

,GPS Time,Device Time,Longitude,Latitude,GPS Speed (Meters/second),Horizontal Dilution of Precision,Altitude,Bearing,G(x),G(y),G(z),G(calibrated),Engine RPM(rpm),Speed (OBD)(km/h),Acceleration Sensor(Total)(g),Litres Per 100 Kilometer(Instant)(l/100km),Speed (GPS)(km/h),Acceleration Sensor(Z axis)(g)
0,Fri Mar 25 17:18:15 GMT+03:00 2016,25-Mar-2016 17:18:08.672,37.583105,55.848558,1.0,14.0,143.0,0.0,-1.069122,8.187592,4.949341,0.002015,-,0.0,0.002015,-,3.5999999046325684,0.405627
1,Fri Mar 25 17:18:16 GMT+03:00 2016,25-Mar-2016 17:18:08.767,37.583102,55.848554,1.0,14.0,143.0,0.0,-1.529938,8.353485,5.078201,0.027657,-,0.0,0.027657,-,3.5999999046325684,0.417656
2,Fri Mar 25 17:18:16 GMT+03:00 2016,25-Mar-2016 17:18:08.867,37.583102,55.848554,1.0,14.0,143.0,0.0,-1.483780,8.233429,4.653717,0.016112,-,0.0,0.016112,-,3.5999999046325684,0.374385
3,Fri Mar 25 17:18:16 GMT+03:00 2016,25-Mar-2016 17:18:08.966,37.583102,55.848554,1.0,14.0,143.0,0.0,-1.489288,8.442078,4.666107,0.015226,-,0.0,0.015226,-,3.5999999046325684,0.375648
4,Fri Mar 25 17:18:16 GMT+03:00 2016,25-Mar-2016 17:18:09.067,37.583102,55.848554,1.0,14.0,143.0,0.0,-0.720825,8.725891,4.743317,0.034077,-,0.0,0.034077,-,3.5999999046325684,0.383519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22276,Fri Mar 25 17:55:28 GMT+03:00 2016,25-Mar-2016 17:55:21.259,37.525679,55.738373,0.0,4.0,166.0,0.0,0.998703,9.563507,1.149261,0.006151,599.5,0.0,0.006151,-,0.0,0.017152
22277,Fri Mar 25 17:55:28 GMT+03:00 2016,25-Mar-2016 17:55:21.358,37.525679,55.738373,0.0,4.0,166.0,0.0,1.070480,9.641998,1.154083,0.006844,601.25,0.0,0.006844,-,0.0,0.017644
22278,Fri Mar 25 17:55:28 GMT+03:00 2016,25-Mar-2016 17:55:21.458,37.525679,55.738373,0.0,4.0,166.0,0.0,1.026428,9.537384,1.169113,0.006450,601.25,0.0,0.006450,-,0.0,0.019176
22279,Fri Mar 25 17:55:28 GMT+03:00 2016,25-Mar-2016 17:55:21.557,37.525679,55.738373,0.0,4.0,166.0,0.0,0.681564,8.894714,1.603546,-0.057069,609.0,0.0,-0.057069,-,0.0,0.063460


In [3]:
# Достаем данные из таблицы

time_GPS = df.iloc[:, 0].values.tolist()
time_OBD = df.iloc[:, 1].values.tolist()
time_GPS = [x[11:19] for x in time_GPS]
time_OBD = [x[12:] for x in time_OBD]
time_GPS = [int(x[:2])*3600 + int(x[3:5])*60 + int(x[6:]) - 62295 for x in time_GPS] # перевод в секунды
time_OBD = [int(x[:2])*3600 + int(x[3:5])*60 + float(x[6:]) - 62288.672 for x in time_OBD] # перевод в секунды


Longitude = df.iloc[:, 2].values.tolist() # думаю, что это x
Latitude = df.iloc[:, 3].values.tolist() # думаю, что это y
Longitude = [float(x) * 63.995 * 1000 - 2405130.8281531497 for x in Longitude] # перевод в метры (не уверен)
Latitude = [float(x) * 111.1 * 1000 - 6204774.826019 for x in Latitude] # перевод в метры (не уверен)

speed_OBD = df.iloc[:, 13].values.tolist() # km/h
speed_GPS = df.iloc[:, 16].values.tolist() # km/h

speed_OBD = [float(x) for x in speed_OBD]
speed_GPS = [float(x) if x != '-' else 0 for x in speed_GPS] # '-' мб надо оставить

Bearing = df.iloc[:, 7].values.tolist()

In [4]:
# Разбиение скоростей на x, y

speed_OBD_x = np.array([])
speed_OBD_y = np.array([])

speed_GPS_x = np.array([])
speed_GPS_y = np.array([])

for i in range(len(Bearing)):
    V = speed_GPS[i]
    Vx = V * np.cos(Bearing[i])
    Vy = V * np.sin(Bearing[i])
    speed_GPS_x = np.append(speed_GPS_x, Vx)
    speed_GPS_y = np.append(speed_GPS_y, Vy)

    V = speed_OBD[i]
    Vx = V * np.cos(Bearing[i])
    Vy = V * np.sin(Bearing[i])
    speed_OBD_x = np.append(speed_OBD_x, Vx)
    speed_OBD_y = np.append(speed_OBD_y, Vy)

# Попытки разабраться на единичках и ноликах:

In [2]:
# Фаза Predict

# Пусть Х = [x,y,x*,y*]

# ДО:
X = np.array([0, 1, 0, 1])

# ТЕКУЩЕЕ СОСТОЯНИЕ:
dt = 1
#         xk = x  y  x* y*
F = np.array([[1, 0, dt, 0],  # x
              [0, 1, 0, dt],  # y
              [0, 0, 1, 0],  # x*
              [0, 0, 0, 1]]) # y*

# ПРЕДСКАЗАНИЕ:
#         xk = x  y  x* y*
# B = np.array([[1, 0, 1, 0],  # x
#               [0, 1, 0, 1],  # y
#               [0, 0, 1, 0],  # x*
#               [0, 0, 0, 1]]) # y*
B = np.eye(4) * dt # матрица включения того или иного параметра вектора U
U = np.array([0, 0, 0, 1]) # Изменение в параметрах (если 0 - нет изменения, а если не ноль, то насколько изменился)


X_k = F @ X.T + B @ U.T

# -----------------------------------

P = np.eye(4) * 100

# ковариация шума процесса:
Q = np.array([[1, 0, 0, 0],  # x
              [0, 1, 0, 0],  # y
              [0, 0, 100, 0],  # x*
              [0, 0, 0, 100]]) # y*

Pk = F @ P @ F.T + Q

print(F @ X.T)
print(X_k)

[0 2 0 1]
[0. 2. 0. 2.]


In [3]:
B = np.eye(4)  # матрица включения
U = np.array([0, 0, 0, 1])

B = B @ U.T
B

array([0., 0., 0., 1.])

In [4]:
from filterpy.common import Q_discrete_white_noise

Q = Q_discrete_white_noise(4, var=10.)
print(Q)

[[ 0.27777778  0.83333333  1.66666667  1.66666667]
 [ 0.83333333  2.5         5.          5.        ]
 [ 1.66666667  5.         10.         10.        ]
 [ 1.66666667  5.         10.         10.        ]]
